In [26]:
import cartopy.crs as ccrs
import fsspec
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import planetary_computer
import pystac_client
import xarray as xr

In [27]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [28]:
longitude = 79.109
latitude = 22.746

geometry = {
    "type": "Point",
    "coordinates": [longitude, latitude],
}

search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    intersects=geometry,
    datetime="2023-04-02/2023-04-03",
    query={"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
)
items = list(search.items())

print(f"Found {len(items)} items:")
print(items[0].assets['ch4'].href)

Found 2 items:
https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__CH4___/2023/04/03/S5P_OFFL_L2__CH4____20230403T063304_20230403T081434_28345_03_020500_20230404T225423/S5P_OFFL_L2__CH4____20230403T063304_20230403T081434_28345_03_020500_20230404T225423.nc?st=2023-09-02T07%3A24%3A32Z&se=2023-09-04T07%3A24%3A32Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-09-03T07%3A24%3A31Z&ske=2023-09-10T07%3A24%3A31Z&sks=b&skv=2021-06-08&sig=qsv1oNGgdzU3CFRer5i0oXBiFzcuNIl1u3ptVCmO3sM%3D


In [30]:
f = fsspec.open(items[0].assets["ch4"].href).open()
ds = xr.open_dataset(f, group="PRODUCT", engine="h5netcdf")
ds